1. Configuración Inicial y Librerías
Explicación:
Importamos las librerías necesarias para el análisis y procesamiento de datos
pandas: Para manipulación de datos
numpy: Para operaciones numéricas
seaborn y matplotlib: Para visualizaciones
datetime: Para manejo de fechas
Configuramos el estilo de las gráficas para mejor visualización

In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

plt.style.use('default')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 8)

2. Carga de Datos
Explicación:
Cargamos el dataset desde el archivo CSV
Mostramos las primeras 5 filas para verificar la carga correcta

In [169]:
df = pd.read_csv('../data/chatbot_satisfaction_dataset_utf8.csv')
df.head()

,id_estudiante,fecha_consulta,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,id_asesor,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta
0,E00001,2024-09-20,True,Solicitud de documentos,Media,2436,1159,A215,1,False,5,-2,3,septiembre
1,E00002,2023-02-07,False,Información general,Baja,2699,1163,A192,3,True,5,19,4,febrero
2,E00003,2024-10-04,True,Información general,Media,1062,434,A242,2,False,4,48,2,octubre
3,E00004,2023-10-24,True,Solicitud de documentos,Baja,663,620,A007,3,True,2,88,3,octubre
4,E00005,2024-12-31,True,Solicitud de documentos,Baja,2376,164,A259,1,True,4,23,4,diciembre


3. Ingeniería de Características Temporales
Explicación:
Convertimos la columna de fecha a formato datetime para poder extraer componentes temporales
Extraemos el año de la consulta
Extraemos el mes de la consulta
Extraemos el día de la semana (0-6, donde 0 es lunes)
Extraemos el trimestre del año
Creamos una variable binaria que indica si la consulta fue en fin de semana

In [170]:
# Convertir fecha_consulta a datetime
df['fecha_consulta'] = pd.to_datetime(df['fecha_consulta'])

# Extraer características temporales
df['año_consulta'] = df['fecha_consulta'].dt.year
df['mes_consulta'] = df['fecha_consulta'].dt.month
df['dia_semana'] = df['fecha_consulta'].dt.dayofweek

# Crear variable binaria para fin de semana
df['es_fin_semana'] = df['dia_semana'].isin([5, 6]).astype(int)
df.head()

,id_estudiante,fecha_consulta,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,id_asesor,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana
0,E00001,2024-09-20,True,Solicitud de documentos,Media,2436,1159,A215,1,False,5,-2,3,9,2024,4,0
1,E00002,2023-02-07,False,Información general,Baja,2699,1163,A192,3,True,5,19,4,2,2023,1,0
2,E00003,2024-10-04,True,Información general,Media,1062,434,A242,2,False,4,48,2,10,2024,4,0
3,E00004,2023-10-24,True,Solicitud de documentos,Baja,663,620,A007,3,True,2,88,3,10,2023,1,0
4,E00005,2024-12-31,True,Solicitud de documentos,Baja,2376,164,A259,1,True,4,23,4,12,2024,1,0


In [171]:
df.drop('id_estudiante', axis=1, inplace=True)
df.drop('fecha_consulta', axis=1, inplace=True)
df.drop('id_asesor', axis=1, inplace=True)


4. Ingeniería de Características de Tiempo de Servicio
   Explicación:
Calculamos el tiempo total de atención sumando tiempo de consulta y espera
Creamos un ratio que muestra la proporción entre tiempo de espera y tiempo de consulta
Categorizamos el tiempo de espera en 4 niveles usando cuartiles

In [172]:
# Calcular tiempo total de atención
df['tiempo_total_atencion'] = df['tiempo_consulta'] + df['tiempo_espera']

# Calcular ratio de tiempo de espera vs tiempo de consulta
df['ratio_tiempo_espera_consulta'] = df['tiempo_espera'] / df['tiempo_consulta']

# Crear categorías de tiempo de espera
df['categoria_tiempo_espera'] = pd.qcut(df['tiempo_espera'], q=4, labels=['Muy Bajo', 'Bajo', 'Alto', 'Muy Alto'])
df.head()

,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana,tiempo_total_atencion,ratio_tiempo_espera_consulta,categoria_tiempo_espera
0,True,Solicitud de documentos,Media,2436,1159,1,False,5,-2,3,9,2024,4,0,3595,0.475780,Muy Alto
1,False,Información general,Baja,2699,1163,3,True,5,19,4,2,2023,1,0,3862,0.430900,Muy Alto
2,True,Información general,Media,1062,434,2,False,4,48,2,10,2024,4,0,1496,0.408663,Bajo
3,True,Solicitud de documentos,Baja,663,620,3,True,2,88,3,10,2023,1,0,1283,0.935143,Alto
4,True,Solicitud de documentos,Baja,2376,164,1,True,4,23,4,12,2024,1,0,2540,0.069024,Muy Bajo


5. Ingeniería de Características de Calidad y Satisfacción
   Explicación:
Combinamos la calidad del servicio con la resolución de la consulta
Creamos un índice de satisfacción general promediando satisfacción e índice promotor
Calculamos la eficiencia del servicio basada en la resolución y el tiempo total

In [173]:
# Crear variable combinada de calidad y resolución
df['calidad_resolucion'] = df['calidad_del_servicio'] * df['consulta_resuelta'].astype(int)

# Crear variable de satisfacción general
df['satisfaccion_general'] = (df['satisfaccion_estudiante'] + df['indice_promotor_neto']) / 2

# Crear variable de eficiencia del servicio
df['eficiencia_servicio'] = df['consulta_resuelta'].astype(int) / (df['tiempo_total_atencion'] + 1)


df.head()

,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana,tiempo_total_atencion,ratio_tiempo_espera_consulta,categoria_tiempo_espera,calidad_resolucion,satisfaccion_general,eficiencia_servicio
0,True,Solicitud de documentos,Media,2436,1159,1,False,5,-2,3,9,2024,4,0,3595,0.475780,Muy Alto,0,1.5,0.000000
1,False,Información general,Baja,2699,1163,3,True,5,19,4,2,2023,1,0,3862,0.430900,Muy Alto,3,12.0,0.000259
2,True,Información general,Media,1062,434,2,False,4,48,2,10,2024,4,0,1496,0.408663,Bajo,0,26.0,0.000000
3,True,Solicitud de documentos,Baja,663,620,3,True,2,88,3,10,2023,1,0,1283,0.935143,Alto,3,45.0,0.000779
4,True,Solicitud de documentos,Baja,2376,164,1,True,4,23,4,12,2024,1,0,2540,0.069024,Muy Bajo,1,13.5,0.000394


6. Codificación de Variables Categóricas
Explicación:
Convertimos variables categóricas a formato numérico
Usamos one-hot encoding para tipo de consulta y mes
Usamos codificación ordinal para urgencia (1=Baja, 2=Media, 3=Alta)

In [174]:
cat_cols = ['tipo_consulta', 'urgencia', 'mes_consulta', 'consulta_resuelta', 'derivado']

for col in cat_cols:
    df[col] = df[col].astype('category')

df.info() # Show info about all columns and data types
print("\nFirst few rows:")
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.max_rows', 5) # Show first 5 rows
#display(df)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   derivado                      30000 non-null  category
 1   tipo_consulta                 30000 non-null  category
 2   urgencia                      30000 non-null  category
 3   tiempo_consulta               30000 non-null  int64   
 4   tiempo_espera                 30000 non-null  int64   
 5   calidad_del_servicio          30000 non-null  int64   
 6   consulta_resuelta             30000 non-null  category
 7   satisfaccion_estudiante       30000 non-null  int64   
 8   indice_promotor_neto          30000 non-null  int64   
 9   puntaje_esfuerzo_cliente      30000 non-null  int64   
 10  mes_consulta                  30000 non-null  category
 11  año_consulta                  30000 non-null  int32   
 12  dia_semana                    30000 non-null  

,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana,tiempo_total_atencion,ratio_tiempo_espera_consulta,categoria_tiempo_espera,calidad_resolucion,satisfaccion_general,eficiencia_servicio
0,True,Solicitud de documentos,Media,2436,1159,1,False,5,-2,3,9,2024,4,0,3595,0.475780,Muy Alto,0,1.5,0.000000
1,False,Información general,Baja,2699,1163,3,True,5,19,4,2,2023,1,0,3862,0.430900,Muy Alto,3,12.0,0.000259
2,True,Información general,Media,1062,434,2,False,4,48,2,10,2024,4,0,1496,0.408663,Bajo,0,26.0,0.000000
3,True,Solicitud de documentos,Baja,663,620,3,True,2,88,3,10,2023,1,0,1283,0.935143,Alto,3,45.0,0.000779
4,True,Solicitud de documentos,Baja,2376,164,1,True,4,23,4,12,2024,1,0,2540,0.069024,Muy Bajo,1,13.5,0.000394


7. Creación de Características de Interacción
Explicación:
Creamos variables de interacción entre diferentes características
Estas interacciones pueden capturar relaciones no lineales entre variables

In [175]:
# Interacción entre urgencia y tiempo de espera
#df['urgencia_tiempo_espera'] = df['urgencia_encoded'] * df['tiempo_espera']

# Interacción entre calidad y tiempo de consulta
df['calidad_tiempo_consulta'] = df['calidad_del_servicio'] * df['tiempo_consulta']

# Interacción entre derivado y tiempo total
df['derivado_tiempo_total'] = df['derivado'].astype(int) * df['tiempo_total_atencion']
df.head()

,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana,tiempo_total_atencion,ratio_tiempo_espera_consulta,categoria_tiempo_espera,calidad_resolucion,satisfaccion_general,eficiencia_servicio,calidad_tiempo_consulta,derivado_tiempo_total
0,True,Solicitud de documentos,Media,2436,1159,1,False,5,-2,3,9,2024,4,0,3595,0.475780,Muy Alto,0,1.5,0.000000,2436,3595
1,False,Información general,Baja,2699,1163,3,True,5,19,4,2,2023,1,0,3862,0.430900,Muy Alto,3,12.0,0.000259,8097,0
2,True,Información general,Media,1062,434,2,False,4,48,2,10,2024,4,0,1496,0.408663,Bajo,0,26.0,0.000000,2124,1496
3,True,Solicitud de documentos,Baja,663,620,3,True,2,88,3,10,2023,1,0,1283,0.935143,Alto,3,45.0,0.000779,1989,1283
4,True,Solicitud de documentos,Baja,2376,164,1,True,4,23,4,12,2024,1,0,2540,0.069024,Muy Bajo,1,13.5,0.000394,2376,2540


8. Normalización de Variables Numéricas
Explicación:
Normalizamos las variables numéricas para que estén en el rango [0,1]
Esto ayuda a que todas las variables tengan la misma escala
Mejora el rendimiento de los algoritmos de machine learning

In [176]:
from sklearn.preprocessing import MinMaxScaler

# Seleccionar variables numéricas para normalizar
numeric_features = ['tiempo_consulta', 'tiempo_espera', 'tiempo_total_atencion', 
                   'ratio_tiempo_espera_consulta', 'eficiencia_servicio']

# Aplicar normalización Min-Max
scaler = MinMaxScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])
df.head()

,derivado,tipo_consulta,urgencia,tiempo_consulta,tiempo_espera,calidad_del_servicio,consulta_resuelta,satisfaccion_estudiante,indice_promotor_neto,puntaje_esfuerzo_cliente,mes_consulta,año_consulta,dia_semana,es_fin_semana,tiempo_total_atencion,ratio_tiempo_espera_consulta,categoria_tiempo_espera,calidad_resolucion,satisfaccion_general,eficiencia_servicio,calidad_tiempo_consulta,derivado_tiempo_total
0,True,Solicitud de documentos,Media,0.674138,0.966499,1,False,5,-2,3,9,2024,4,0,0.748258,0.475039,Muy Alto,0,1.5,0.000000,2436,3595
1,False,Información general,Baja,0.747829,0.969849,3,True,5,19,4,2,2023,1,0,0.804646,0.430096,Muy Alto,3,12.0,0.013720,8097,0
2,True,Información general,Media,0.289157,0.359296,2,False,4,48,2,10,2024,4,0,0.304963,0.407827,Bajo,0,26.0,0.000000,2124,1496
3,True,Solicitud de documentos,Baja,0.177361,0.515075,3,True,2,88,3,10,2023,1,0,0.259979,0.935052,Alto,3,45.0,0.041277,1989,1283
4,True,Solicitud de documentos,Baja,0.657327,0.133166,1,True,4,23,4,12,2024,1,0,0.525449,0.067708,Muy Bajo,1,13.5,0.020858,2376,2540


9. Guardado del Dataset Procesado
Explicación:
Guardamos el dataset procesado con todas las nuevas características
El archivo se guarda en formato CSV
No incluimos el índice en el archivo guardado
¿Te gustaría que profundice en alguna de estas secciones o que te explique algún concepto específico con más detalle?

In [177]:
df['satisfaccion'] = df['satisfaccion_estudiante'].replace({
    1: 1, 
    2: 1,
    3: 2,
    4: 3, 
    5: 3
})
df.drop('satisfaccion_estudiante', axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   derivado                      30000 non-null  category
 1   tipo_consulta                 30000 non-null  category
 2   urgencia                      30000 non-null  category
 3   tiempo_consulta               30000 non-null  float64 
 4   tiempo_espera                 30000 non-null  float64 
 5   calidad_del_servicio          30000 non-null  int64   
 6   consulta_resuelta             30000 non-null  category
 7   indice_promotor_neto          30000 non-null  int64   
 8   puntaje_esfuerzo_cliente      30000 non-null  int64   
 9   mes_consulta                  30000 non-null  category
 10  año_consulta                  30000 non-null  int32   
 11  dia_semana                    30000 non-null  int32   
 12  es_fin_semana                 30000 non-null  

In [178]:
# Guardar dataset procesado
# El parámetro index=False indica que no se incluirá la columna de índices numéricos
# que pandas agrega automáticamente (0,1,2,3...) al guardar el archivo CSV.
# Solo se guardarán las columnas con los datos reales del DataFrame.
df.to_csv('../data/chatbot_satisfaction_dataset_engineered.csv', index=False)
print("Dataset guardado exitosamente con las nuevas características")

Dataset guardado exitosamente con las nuevas características
